In [1]:
from poker_game import Deck, Card

deck = Deck()  # Reproducible shuffling
cards = deck.deal(5)  # Deal 5 cards
print(cards)  # [Ah, Kd, Qs, Jc, Ts]

[Ten Of Heart, Five Of Club, Two Of Diamond, Six Of Diamond, Three Of Diamond]


In [2]:
from poker_game import HandEvaluator, Card

cards = [Card.from_string(s) for s in ['Ah', 'Ad', 'Kh', 'Kd', 'Qs']]
hand_rank, tiebreakers = HandEvaluator.evaluate_hand(cards)
description = HandEvaluator.get_hand_description(cards)
print(description)  # "Two Pair"

Two Pair


In [ ]:
from poker_game import GameState, PlayerState, Action

player = PlayerState(player_id=0, stack=100.0)
player.bet(10.0)  # Place a bet
print(player.stack)  # 90.0

190.0


In [4]:
from poker_game import PokerGame, Action

# Create a game
game = PokerGame(num_players=2, starting_stack=100.0, small_blind=0.5, big_blind=1.0)

# Start a hand
state = game.reset()

# Get valid actions for current player
player = state.current_player()
valid_actions = state.get_valid_actions(player)

# Execute an action
new_state, hand_complete, result = game.step(Action.RAISE, amount=6.0)

# Print game state
print(game.get_game_state_string())

Hand #1 - PREFLOP
Board: (no cards yet)
Pot: $8.0

Players:
  BB (P0): $99.0 (ACTIVE) [Five Of Club Ace Of Spade]
  SB (P1): $93.0 (ACTIVE) [Six Of Diamond King Of Diamond]

Action on: BB (P0)
Current bet: $7.0


In [5]:
# Method 1: Use state.current_player()
current_player = state.current_player()
print(f"Current player: {current_player.player_id} ({current_player.position})")

# Method 2: Use state.current_player_idx
print(f"Current player index: {state.current_player_idx}")

# Method 3: Check if betting round is complete
if not state.is_betting_round_complete():
    print("It's someone's turn - action required")
else:
    print("Betting round is complete - no action needed")

# Method 4: Get valid actions for current player
valid_actions = state.get_valid_actions(current_player)
print(f"Valid actions for current player: {[a.value for a in valid_actions]}")

Current player: 0 (BB)
Current player index: 0
It's someone's turn - action required
Valid actions for current player: ['fold', 'call', 'raise']


In [6]:
# Test LLM prompt generation
print(f"Current betting round: {state.betting_round}")
print(f"Action history: {state.action_history}")
print()

prompt = state.get_llm_prompt(player_perspective=0)
print("LLM Prompt:")
print(prompt)

Current betting round: BettingRound.PREFLOP
Action history: [{'player_id': 1, 'action': 'bet', 'amount': 0.5, 'betting_round': 'preflop', 'pot_after': 0.5, 'is_blind': True}, {'player_id': 0, 'action': 'bet', 'amount': 1.0, 'betting_round': 'preflop', 'pot_after': 1.5, 'is_blind': True}, {'player_id': 1, 'action': 'raise', 'amount': 6.0, 'betting_round': 'preflop', 'pot_after': 8.0, 'is_blind': False}]

LLM Prompt:
You are a specialist in playing heads-up No Limit Texas Holdem. The following will be a game scenario and you need to make the optimal decision.

Here is a game summary:

The small blind is 0.5 chips and the big blind is 1.0 chips. Everyone started with 100.0 chips.
The player positions involved in this game are SB, BB.
In this hand, your position is BB, and your holding is [Five Of Club and Ace Of Spade].
Before the flop, SB raise 6.0 chips. Assume that all other players that is not mentioned folded.

Now it is your turn to make a move.
To remind you, the current pot size i

In [13]:
# Test 6-player game
from poker_game import PokerGame

# Create a 6-player game
game6 = PokerGame(num_players=6, starting_stack=100.0, small_blind=0.5, big_blind=1.0)
state6 = game6.reset()

print('6-handed poker:')
print(f'Button position: {state6.button_position}')
print('Player positions:')
for i, player in enumerate(game6.players):
    print(f'  Player {i}: {player.position}')
print('Action history:')
for action in state6.action_history:
    player_pos = game6.players[action['player_id']].position
    print(f'  {player_pos}: {action["action"]} {action["amount"]}')
print(f'Current player: {state6.current_player().position} (P{state6.current_player_idx})')
print()
print('Game state:')
print(game6.get_game_state_string())

6-handed poker:
Button position: 1
Player positions:
  Player 0: CO
  Player 1: BTN
  Player 2: SB
  Player 3: BB
  Player 4: UTG
  Player 5: HJ
Action history:
  SB: bet 0.5
  BB: bet 1.0
Current player: UTG (P4)

Game state:
Hand #1 - PREFLOP
Board: (no cards yet)
Pot: $1.5

Players:
  CO (P0): $100.0 (ACTIVE) [Ten Of Spade Jack Of Diamond]
  BTN (P1): $100.0 (ACTIVE) [Five Of Club Eight Of Club]
  SB (P2): $99.5 (ACTIVE) [Jack Of Spade Seven Of Spade]
  BB (P3): $99.0 (ACTIVE) [Five Of Spade Five Of Diamond]
  UTG (P4): $100.0 (ACTIVE) [Eight Of Spade Five Of Heart]
  HJ (P5): $100.0 (ACTIVE) [Four Of Heart Queen Of Spade]

Action on: UTG (P4)
Current bet: $1.0


In [8]:
print('Direct stack check:')
for i, player in enumerate(game6.players):
    print(f'  Player {i} ({player.position}): ${player.stack:.1f}')

Direct stack check:
  Player 0 (CO): $100.0
  Player 1 (BTN): $100.0
  Player 2 (SB): $99.5
  Player 3 (BB): $99.0
  Player 4 (UTG): $100.0
  Player 5 (HJ): $100.0
